In [1]:
import nltk
import pandas as pd
from corextopic import corextopic as ct

In [2]:
df=pd.read_csv("C:\\Users\\lenovo\\Desktop\\IR\\Project\\dist-filtered-preprocessed.csv")

In [3]:
df.head()

,Unnamed: 0,lemmas_back_to_text,lemma_tokens,Labels
0,0,social distancing new buzz word combat coronav...,"['social', 'distancing', 'new', 'buzz', 'word'...",NaN
1,1,feel social distancing coronavirus demonstrate...,"['feel', 'social', 'distancing', 'coronavirus'...",1.0
2,2,insane post @thecut wear bra home self distanc...,"['insane', 'post', 'thecut', 'wear', 'bra', 'h...",0.0
3,3,scary reminder importance social distancing sl...,"['scary', 'reminder', 'importance', 'social', ...",1.0
4,4,social distance great important way number way...,"['social', 'distance', 'great', 'important', '...",1.0


In [4]:
df=df.drop(columns=["Unnamed: 0"])

In [5]:
df.lemmas_back_to_text=df.lemmas_back_to_text.astype("str")

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5531 entries, 0 to 5530
Data columns (total 3 columns):
lemmas_back_to_text    5531 non-null object
lemma_tokens           5531 non-null object
Labels                 9 non-null float64
dtypes: float64(1), object(2)
memory usage: 129.7+ KB


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(
    max_df=.5,
    min_df=10,
    max_features=None,
    ngram_range=(1, 2),
    norm=None,
    binary=True,
    use_idf=False,
    sublinear_tf=False
)
vectorizer = vectorizer.fit(df['lemmas_back_to_text'])
tfidf = vectorizer.transform(df['lemmas_back_to_text'])
vocab = vectorizer.get_feature_names()
print(len(vocab))

927


In [8]:
vocab

['000',
 '10',
 '100',
 '12',
 '19',
 '19 case',
 '19 pandemic',
 '19 rule',
 '19 social',
 '1918',
 '20',
 '2020',
 '2022',
 '30',
 '50',
 'abide',
 'able',
 'absolutely',
 'accord',
 'act',
 'action',
 'activity',
 'actually',
 'add',
 'adhere',
 'adhere social',
 'advice',
 'age',
 'ago',
 'agree',
 'ahead',
 'ai',
 'air',
 'allow',
 'amazing',
 'america',
 'american',
 'americanair',
 'amid',
 'amid coronavirus',
 'amp',
 'answer',
 'anymore',
 'apart',
 'apparently',
 'appear',
 'apply',
 'april',
 'ar',
 'area',
 'arrest',
 'article',
 'ask',
 'ass',
 'attend',
 'available',
 'avoid',
 'away',
 'baby',
 'bad',
 'bar',
 'base',
 'beach',
 'beautiful',
 'begin',
 'behavior',
 'believe',
 'biden',
 'big',
 'bike',
 'birthday',
 'bit',
 'black',
 'blame',
 'bless',
 'break',
 'break social',
 'briefing',
 'bring',
 'bus',
 'business',
 'california',
 'call',
 'cancel',
 'capacity',
 'car',
 'care',
 'case',
 'cat',
 'catch',
 'cause',
 'cdc',
 'cdcgov',
 'celebrate',
 'celebration',


In [20]:
anchors = []
model = ct.Corex(n_hidden=10, seed=42)
model = model.fit(
    tfidf,
    words=vocab
)

In [21]:
for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))

Topic #1: covid 19, 19, covid, 19 social, covid19, 19 case, distancing covid, spread covid, 19 pandemic, new york
Topic #2: mask, wear, wear mask, mask social, distancing wear, distancing mask, mask wear, face mask, wear social, people wear
Topic #3: practice, practice social, hand, wash, wash hand, mask practice, distancing wash, washing, mask wash, continue practice
Topic #4: distance, social distance, mask distance, distance mask, socially distance, socially, mask socially, distance wear, distance good, ft distance
Topic #5: guideline, home, stay home, stay, distancing guideline, distancing stay, follow social, follow, order, home order
Topic #6: physical, physical distancing, ft, apart, spread, ft apart, stop, quarantine, stay ft, quarantine social
Topic #7: measure, coronavirus, coronavirus social, distancing measure, distancing coronavirus, ignore, death, relax, protest, week
Topic #8: rule, distancing rule, pandemic, need social, enforce, enforce social, non, amid, official, cor

In [23]:
anchor=[
    ['wear',"mask","mask distancing","covering"], #face mask
    ["distnace", "social distancing","mandate","measure","physcial distancing","physcial"] # distnacing 
    #goverment
]
anchors = [
    [a for a in topic if a in vocab]
    for topic in anchors
]

model = ct.Corex(n_hidden=10, seed=42)
model = model.fit(
    tfidf,
    words=vocab,
    anchors=anchors, # Pass the anchors in here
    anchor_strength=3 # Tell the model how much it should rely on the anchors
)

In [24]:
for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))

Topic #1: practice, practice social, hand, wash, wash hand, mask practice, distancing wash, washing, mask wash, continue practice
Topic #2: measure, coronavirus, coronavirus social, distancing measure, distancing coronavirus, ignore, death, relax, protest, week
Topic #3: rule, distancing rule, pandemic, need social, enforce, enforce social, non, amid, official, coronavirus pandemic
Topic #4: guideline, home, stay home, stay, distancing guideline, distancing stay, follow social, follow, order, home order
Topic #5: distance, social distance, mask distance, distance mask, socially distance, socially, mask socially, distance wear, distance good, ft distance
Topic #6: covid 19, 19, covid, 19 social, covid19, 19 case, distancing covid, spread covid, 19 pandemic, new york
Topic #7: physical, physical distancing, ft, apart, spread, ft apart, stop, quarantine, stay ft, quarantine social
Topic #8: trump, people, die, rally, understand, people social, break, white, fauci, stupid
Topic #9: case, l

In [27]:
topic_df = pd.DataFrame(
    model.transform(tfidf), 
    columns=["topic_{}".format(i+1) for i in range(10)]
).astype(float)
topic_df.index = df.index
df = pd.concat([df, topic_df], axis=1)


In [28]:
df[df.topic_2 == 1]

ValueError: cannot reindex from a duplicate axis